In [1]:
import pandas as pd
import os
import pickle
from docx import Document
import re

In [2]:
path='../rawdata'

In [3]:

for key in os.listdir(path):
    print(key)

已查_民事案件201811-201910 （渝北、江北）.xlsx
已查_民事案件2015-2017.xlsx
1.py
已查_16-17年行政刑事案件.xlsx
两法院民事案由分值.xlsx
.~lock.案件列表201601-201712行政、刑事 .xlsx#
已查_案件列表201801-201910行政、刑事.xlsx
民事18_19
两院法官人员信息.xlsx
已查_民事案件201801-10（渝北、江北）.xlsx
.~lock.两院法官人员信息.xlsx#
数据清洗后
行政＋刑事18_19
.~lock.民事案件201811-201910 （渝北、江北）.xlsx#
已查_案件列表2015行政、刑事.xlsx
行政＋刑事15_17
.ipynb_checkpoints


## step1:4,5,6为案由计算

In [4]:
anyou=pd.read_excel(os.path.join(path,'两法院民事案由分值.xlsx')) 
anyou0=anyou[anyou['法院']=='重庆市江北区人民法院']
anyou1=anyou[anyou['法院']=='重庆市渝北区人民法院']

In [5]:
def anyou2id(anyou0):
    anyoudict0={}
    for index, row in anyou0.iterrows():
        anyoudict0[row[4]]=row[1]
    return anyoudict0

In [6]:
anyoudict0=anyou2id(anyou0)
anyoudict1=anyou2id(anyou1)

In [7]:
anyou.head(1)

,法院,分值,案由代码,上级代码,案由名称,案件类型,启用状态
0,重庆市江北区人民法院,6,9000,0,民事案由,民事,正常启用


## step2:去掉法官重名的情况，将人名与Id一一对应

In [8]:
judge=pd.read_excel(os.path.join(path,'两院法官人员信息.xlsx')) 

In [9]:
judge.head()

,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官
0,1,杨青,重庆市江北区人民法院,执行局,NaT,NaN,女,NaN,1996-01-22,NaN,2.0
1,2,凌文英,重庆市江北区人民法院,民事审判一庭,1985-12-20,审判员,女,NaN,1967-07-04,四级高级法官,1.0
2,3,张雪琪,重庆市江北区人民法院,执行局,NaT,NaN,女,NaN,1995-03-21,NaN,NaN
3,4,张敬,重庆市江北区人民法院,民事审判三庭,2001-07-04,审判员,女,副主任科员,1978-12-01,一级法官,1.0
4,5,汪涓,重庆市江北区人民法院,监察室,1986-07-01,审判员,女,副调研员,1964-03-06,四级高级法官,2.0


In [10]:
judge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 11 columns):
id        972 non-null int64
姓名        972 non-null object
法院        972 non-null object
部门        972 non-null object
入院日期      477 non-null datetime64[ns]
法律职务      476 non-null object
性别        972 non-null object
行政职务      315 non-null object
出生日期      972 non-null datetime64[ns]
等级        210 non-null object
是否员额法官    635 non-null float64
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 83.6+ KB


In [11]:
judge0=judge[judge['法院']=='重庆市江北区人民法院']
judge1=judge[judge['法院']=='重庆市渝北区人民法院']

In [12]:
#选出重复的名字：重庆市江北区人民法院：删除id==370
#渝北区人民法院：删除id==501,639,648,664,673,737,765,797,836
judge0[judge0['姓名'].duplicated(False)]

,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官
369,370,陈瑶,重庆市江北区人民法院,司法警察大队,NaT,NaN,男,NaN,1995-01-13,NaN,2.0
384,385,陈瑶,重庆市江北区人民法院,金融审判庭,NaT,NaN,女,NaN,1987-11-16,NaN,NaN


In [13]:
judge0.drop([369],inplace=True)
judge0[judge0['姓名'].duplicated(False)]

/home/huipu/.local/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官


In [14]:
judge1[judge1['姓名'].duplicated(False)]

,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官
468,469,杨琼,重庆市渝北区人民法院,民事审判二庭,1994-04-01,庭长,女,庭长,1970-05-23,四级高级法官,1.0
500,501,何霜,重庆市渝北区人民法院,民事审判二庭,NaT,NaN,女,NaN,1989-10-18,NaN,NaN
638,639,陈果,重庆市渝北区人民法院,环境资源审判庭,NaT,NaN,女,NaN,1994-12-19,NaN,NaN
647,648,王婷婷,重庆市渝北区人民法院,综合办公室,2006-04-26,司法行政人员,女,其他,1983-01-27,NaN,2.0
659,660,陈果,重庆市渝北区人民法院,环境资源审判庭,2011-08-12,审判员,男,其他,1984-10-10,二级法官,1.0
663,664,马瑞,重庆市渝北区人民法院,民事审判三庭,2018-11-07,法官助理,女,其他,1993-12-17,NaN,NaN
672,673,何霜,重庆市渝北区人民法院,民事审判二庭,NaT,NaN,女,NaN,1994-08-18,NaN,2.0
736,737,郭鹏飞,重庆市渝北区人民法院,民事审判一庭,2018-11-08,法官助理,男,其他,1991-08-19,NaN,NaN
764,765,杨琼,重庆市渝北区人民法院,民事审判二庭,NaT,NaN,女,NaN,1991-10-07,NaN,NaN
796,797,马瑞,重庆市渝北区人民法院,民事审判三庭,NaT,NaN,女,NaN,1995-08-31,NaN,NaN


In [15]:
judge1.drop([500,638,647,663,672,736,764,796,835],inplace=True)
judge1[judge1['姓名'].duplicated(False)]

,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官


In [16]:
def judgeToId(judge1):
    judge2id={}
    # id2judge={}
    for index, row in judge1.iterrows():
        judge2id[row[1]]=row[0]
    #     id2judge[row[0]]=row[1]
    return judge2id

In [17]:
judge2id0=judgeToId(judge0)
judge2id1=judgeToId(judge1)

## step3:原被告数与主体类别

In [18]:
#原被告数与主体类别
#0表示自然人，1表示公司
#返回（自然人/公司，数量得分）
def plaintNum(s):
    numS=s.split(',')
    num=len(numS)
    flag=0#0表示自然人，1表示公司
    for key in numS:
        if len(key)>4:
            flag=1
    if num==1:
        return flag, 0
    elif num==2:
        return flag, 0.2
    else:
        return flag,0.5
plaintNum('兴业银行股份有限公司信用卡中心')

(1, 0)

## step4:适用程序

In [19]:
def proce(s):
    if s=='简易程序（小额诉讼）'or s=='简易程序(小额诉讼)':
        return 0
    if s=='简易程序(非小额诉讼)'or s=='简易程序（非小额诉讼）' or s=='简易':
        return 0.5
    if s=='普通程序'or s=='普通':
        return 1

## step5:标的金额

In [20]:
def money(m):
    if m=='\\N':
        m=0
    m=int(float(m))
    if m<=5000:
        return 0
    elif m<=30000:
        return 0.5
    else:
        return 1

## step6:读取excel数据
datadictFinal为excel2018-2019内的案号

In [21]:
root_pre='民事2018.01-2018.10.xlsx'
data_pre=pd.read_excel(os.path.join(path,'数据清洗后',root_pre))   

In [22]:
data_pre.head(1)

,案件标识,法院,案号,立案案由,原告,被告,承办部门,承办人,申请标的,适用程序,...,签收日期,分案日期,报结日期,结案日期,结案方式,裁判文书,法定审理天数,实际审理天数,计算结果,差值判断
0,00077602-304a-4a5c-a069-96e2ae933f11,重庆市江北区人民法院,(2018)渝0105民初31680号,信用卡纠纷,兴业银行股份有限公司信用卡中心,伯春艳,金融审判庭,郭雪妍,50838.4,简易程序（非小额诉讼）,...,2018-08-01,2018-08-01,2018-09-10 00:00:00,2018-09-10,判决,民事判决书(一审简易程序用),92,45,45,0


In [23]:
data_pre0=data_pre[data_pre['法院']=='重庆市江北区人民法院']
data_pre1=data_pre[data_pre['法院']=='重庆市渝北区人民法院']

In [24]:
root='民事2018.11-2019.10.xlsx'
data=pd.read_excel(os.path.join(path,'数据清洗后',root))   

In [25]:
data.head(1)

,案件标识,法院,案号,立案案由,原告,被告,承办部门,承办人,申请标的,适用程序,...,签收日期,分案日期,报结日期,结案日期,结案方式,裁判文书,法定审理天数,实际审理天数,计算结果,差值判断
0,00055d19-b4a4-4672-9469-dc088ca360c7,重庆市江北区人民法院,(2018)渝0105民初34609号,信用卡纠纷,兴业银行股份有限公司信用卡中心,刘祖康,金融审判庭,毛亚男,53984,简易程序（非小额诉讼）,...,2018-10-22,2018-10-22 00:00:00,2018-12-13 00:00:00,2018-12-14,判决,民事判决书（当事人对案件事实没有争议的用）,92,57,57,0


In [26]:
data0=data[data['法院']=='重庆市江北区人民法院']
data1=data[data['法院']=='重庆市渝北区人民法院']

In [27]:
#datadictFinal为excel2018-2019内的案号与案件标识
datadictFinal=[]
datadict=data['案号']
datadict_pre=data_pre['案号']
datadictFinal=pd.concat([datadict_pre,datadict]).tolist();

In [28]:
print(len(datadict_pre),len(datadict),len(datadictFinal))

15027 22867 37894


## step7:将文书集中案件标识与案号关联:keyDict

In [29]:
#将案件标识与案号关联:keyDict
#root为文书存放目录名，path为保存keyDict的文件名
def signDict(rootw,path1):
    pathn='keyDict.pkl'
    keyDict={}#将案件标识与案号关联
    table=[]#以表格形式存在的文书
    notFindDoc = []#不是以docx结尾的文书（格式转换未成功的文书）
    if os.path.exists(os.path.join(rootw,pathn)):
        print(pathn,'存在')
        read_file = open(os.path.join(rootw,pathn), 'rb')
        keyDict = pickle.load(read_file)
        read_file.close()
        read_file = open(os.path.join(rootw,'table.pkl'), 'rb')
        table = pickle.load(read_file)
        read_file.close()
        read_file = open(os.path.join(rootw,'notFindDoc.pkl'), 'rb')
        notFindDoc = pickle.load(read_file)
        read_file.close()
    else:
        path2=os.path.join(path1,'民事18_19')
        for f in os.listdir(path2):
            root2=os.path.join(path2, f)
            for key_ in os.listdir(root2) :
                key = os.path.join(root2,key_)
                try:
                    if key.endswith(".docx"):
                        document = Document(key)
                        paras = list()
                        for p in document.paragraphs:
                            if p.text.strip() != "":
                                paras.append(p.text.strip())
                        # if(len(paras)==0):
                        #     tables = document.tables
                        #     for t in tables:
                        #         for r in t.rows:
                        #             for c in r.cells:
                        #                 for p in c.paragraphs:
                        #                     if p.text.strip() != "":
                        #                         paras.append(p.text.strip())
                        keyDict[key_.split('.')[0]]=paras[2]
                    else:
                        notFindDoc.append(key)
                except:
                    table.append(key)
        output = open(os.path.join(rootw,pathn), 'wb')
        pickle.dump(keyDict, output,True)
        output.close()
        output = open(os.path.join(rootw,'table.pkl'), 'wb')
        pickle.dump(table, output,True)
        output.close()
        output = open(os.path.join(rootw,'notFindDoc.pkl'), 'wb')
        pickle.dump(notFindDoc, output,True)
        output.close()
    print('keyDict',len(keyDict),'table',len(table),'notFindDoc',len(notFindDoc))
    return keyDict,table,notFindDoc

In [30]:
import os
path='../rawdata'
rootw=os.path.join(path,'数据清洗后','18_19民')
pathn='keyDict.pkl'
keyDict,table,notFindDoc=signDict(rootw,path)

keyDict.pkl 存在
keyDict 42609 table 3785 notFindDoc 1950


In [31]:
keyDict;
#{'M1D_7cc626c2-50c2-4921-a27e-ec10428feaa8': '（2019）渝0112民初9935号','M1D_98cb7509-1656-4804-a762-0e21f1140a0d': '渝0112民初19754号',

## step8:keyDict key与value置换（keyDcitRe），选出在excel和文书都存在的案件==>keyDcitReplace

In [32]:
def keyReplace0(keyDict):
    keyDcitRe={}
    keyDuctSame={}#重复出现的文书要去除
    for key in keyDict:
        if keyDict[key] not in keyDcitRe:
            keyDcitRe[keyDict[key]]=key
        else:
            keyDuctSame[keyDict[key]]=key
    return keyDcitRe,keyDuctSame
def keyReplace(datadictFinal,keyDict):
    print(len(keyDict))
    keyDcitRe,keyDuctSame=keyReplace0(keyDict)
    print('去除掉重复出现的文书后',len(keyDcitRe))#
    keyDcitReplace={}
    biaoshiKey={}
    notExcel={}
    for key in keyDcitRe:
        temp = keyDcitRe[key]
        key=key.replace('（','(')
        key=key.replace('）',')')
        if key in datadictFinal:
            keyDcitReplace[key] = temp
            biaoshiKey[temp]=key
        else:
            notExcel[key]=temp
    print('同在文书集和excel表中的列表',len(keyDcitReplace))
    return keyDcitReplace,biaoshiKey,notExcel

In [33]:
keyDcitReplace,biaoshiKey,notExcel=keyReplace(datadictFinal,keyDict)

42609
去除掉重复出现的文书后 42486
同在文书集和excel表中的列表 31501


In [34]:
print(len(keyDict),len(datadictFinal))#keyDic为文书集中的，datadictFinal为excel2018-2019内的案号

42609 37894


## step9:读取同在excel表和文书集中的18-19年以判决结案的案号与文书内容

In [52]:
import json
import os
from docx import Document
import re
import pickle
#将案件标识与案号关联:keyDict
#root为文书存放目录名，path为保存keyDict的文件名
def signDict(rootw,pathn):
    #保存三个文件：keyDict,table,notFindDoc
    keyDict={}#将案件标识与案号关联
    table=[]#以表格形式存在的文书
    notFindDoc = []#不是以docx结尾的文书（格式转换未成功的文书）
    if os.path.exists(os.path.join(rootw,pathn)):
        read_file = open(os.path.join(rootw,pathn), 'rb')
        keyDict = pickle.load(read_file)
        read_file.close()
        read_file = open(os.path.join(rootw,'table.pkl'), 'rb')
        table = pickle.load(read_file)
        read_file.close()
        read_file = open(os.path.join(rootw,'notFindDoc.pkl'), 'rb')
        notFindDoc = pickle.load(read_file)
        read_file.close()
    else:
        for f in os.listdir(rootw):
            root2=os.path.join(rootw, f)
            for key_ in os.listdir(root2) :
                key = os.path.join(root2,key_)
                try:
                    if key.endswith(".docx"):
                        document = Document(key)
                        paras = list()
                        for p in document.paragraphs:
                            if p.text.strip() != "":
                                paras.append(p.text.strip())
                        # if(len(paras)==0):
                        #     tables = document.tables
                        #     for t in tables:
                        #         for r in t.rows:
                        #             for c in r.cells:
                        #                 for p in c.paragraphs:
                        #                     if p.text.strip() != "":
                        #                         paras.append(p.text.strip())
                        keyDict[key_.split('.')[0]]=paras[2]
                    else:
                        notFindDoc.append(key)
                except:
                    table.append(key)
        output = open(os.path.join(rootw,pathn), 'wb')
        pickle.dump(keyDict, output,True)
        output.close()
        output = open(os.path.join(rootw,'table.pkl'), 'wb')
        pickle.dump(table, output,True)
        output.close()
        output = open(os.path.join(rootw,'notFindDoc.pkl'), 'wb')
        pickle.dump(notFindDoc, output,True)
        output.close()
    print('keyDict',len(keyDict),'table',len(table),'notFindDoc',len(notFindDoc))
    return keyDict,table,notFindDoc


# 读取段落文字
def readPara(doc):  # doc为待读取的文书路径
    document = Document(doc)
    paras=list()
    data=list()
    dang=""
    for p in document.paragraphs:
        if p.text.strip() !="" :
            paras.append(p.text.strip())
            # paras.insert(0,zhuti)
    for i in range(len(paras)):
        if '原告'not in paras[i] and '被告' not in paras[i]:
            data.append(paras[i])
        else:
            paras=paras[i:len(paras)]
            break
    p1 = re.compile('原告.*案')
    # p2=re.compile('被告.*：')
    for i in range(len(paras)):
        if len(p1.findall(paras[i]))==0:
            paras[i]=paras[i].replace("\n", "")
            dang=dang+paras[i].strip()
        else:
            paras=paras[i:len(paras)]
            break
    dang2 = re.split('被告', dang[10:], 1)
    dang2[0] = dang[0:10] + dang2[0]
    data.extend(dang2)
    data.extend(paras)
    return data


#  step 1 将doc处理成字典格式--记录读取的内容到filename
def filter0(root,filename,keyDict,rootw):#root='../rawdata/民事18_19'  rootw='../rawdata/18Data'
    #共保存两个文件：filename,'notread.txt'
    exitNotread=[]#存在在公共集中但读取内容时有问题
    data=[]#目的是为了直观
    NotEixt=[]#没有存在在公共集中
    if os.path.exists(os.path.join(rootw,filename)):
        print(filename+' exists')
        with open(os.path.join(rootw,filename),'r',encoding='utf-8')as f:
            for line in f:
                d=json.loads(line)#转成字典格式
                data.append(d)
        fenr= open(os.path.join(rootw,'notread.txt'), "r",encoding='utf-8')
        exitNotread=[]
        for line in fenr.readlines():
            exitNotread.append(1)
        fenr.close()
    else:
        p1 = re.compile('(.*)事实.*?理由(.*)')
        p2=re.compile('依照(.*)规定')
        for f in os.listdir(root):
            root2=os.path.join(root, f)
            for key in os.listdir(root2) :
                key2 = os.path.join(root2,key)
                datatemp = dict()
#                 temp = key.split('.')[0]
                if key.split('.')[0] in keyDict:
                    try:
                        paras = readPara(key2)
                        if len(paras) == 0:
                            exitNotread.append(key)
                        if len(paras) > 0:
                            datatemp['key']=keyDict[key.split('.')[0]]
                            datatemp['原告'] = paras[3]
                            datatemp['被告'] = paras[4]
                            datatemp['案由+程序'] = paras[5]
#                             print('ok')
                            if '请求' in paras[6]:
                                temp = p1.findall(paras[6])
                                if len(temp) > 0:
                                    temp = temp[0]
                                    if len(temp[1]) > 5:
                                        datatemp['诉讼请求'] = temp[0]
                                        datatemp['事实理由'] = temp[1]
                                    else:
                                        datatemp['诉讼请求'] = paras[6]
                                        datatemp['事实理由'] = paras[7]
                                else:
                                    datatemp['诉讼请求'] = paras[6]
                                    datatemp['事实理由'] = paras[7]
                                if('经审理' in paras[9]):
                                    datatemp['事实理由'] = paras[9]
                                elif('经审理' in paras[8]):
                                    datatemp['事实理由'] = paras[8]
                                temp=p2.findall(paras[12])
                                if(len(temp)>0):
                                    datatemp['适用法条'] = temp[0]
                                else:
                                    datatemp['适用法条'] = ''
                                data.append(datatemp)

                            else:
                                exitNotread.append(key)
#                         break
                    except:
                        exitNotread.append(key)
                else:
#                     print(key.split('.')[0])
                    NotEixt.append(key)
                    
#             break
          
            #记录读取的内容
        with open(os.path.join(rootw,filename), 'w', encoding='utf-8')as f2:
            f2.truncate()  # 清空文件内容
            for key in data:
                json.dump(key, f2, ensure_ascii=False)
                f2.write('\n')

        #记录未读的路径
        with open(os.path.join(rootw,'notread.txt'), "w") as fw:
            fw.truncate()
            for i in range(len(exitNotread)):
                print(str(exitNotread[i]) , file=fw)
        
        print('没有存在于公共集的文书',len(NotEixt))   
                
    print('存在于公共集但读取失败的文书',len(exitNotread))

    print('记录的文书量：',len(data))
    return data

# step 2 将以列表形式存在的json变为一个json存储-->保存在fileE中
def jsonOne(rootw,fileB,fileE):
    if os.path.exists(os.path.join(rootw,fileE)):
        read_file = open(os.path.join(rootw,fileE), 'rb')
        data = pickle.load(read_file)
        read_file.close()
    else:
        with open(os.path.join(rootw,fileB), 'r', encoding='utf-8')as f:
            data = dict()
            for line in f:
                d = json.loads(line)  # 转成字典格式
                k=d['key']
                data[k]= -1
                del d["key"]
                data[k]=d
        output = open(os.path.join(rootw,fileE), 'wb')
        pickle.dump(data, output,True)
        output.close()
    return data

# # if __name__ == '__main__':
#     path = '../rawdata/数据清洗后'
#     rootw = os.path.join(path, '18Data')
# #     pathn = 'keyDict.pkl'
# #     keyDict,table,notFindDoc=signDict(rootw,pathn)
#     data0=filter0(rootw, 'dataRead.json', keyDcitReplace)
#     data=jsonOne('dataRead.json', 'data.json')

In [53]:
keyAll= dict(zip(keyDcitReplace.values(), keyDcitReplace.keys()))#excel和文书中都存在的案件
root='../rawdata/民事18_19'
rootw = '../rawdata/数据清洗后/18_19民'
# os.path.exists(os.path.join(rootw,'dataRead.json'))
# keyDict,table,notFindDoc=signDict(path,rootw,pathn)
content=filter0(root, 'dataRead.json', keyAll,rootw)

dataRead.json exists
存在于公共集但读取失败的文书 848
记录的文书量： 30654


In [79]:
content=jsonOne(rootw,'dataRead.json', 'datajsonOne.json')

In [82]:
content['(2018)渝0112民初30233号']

{'原告': '原告：重庆市金德茧丝绸有限公司，住所地重庆市万州区北山大道1323号，统一社会信用代码915001017428787877。法定代表人：彭中海，执行董事。委托诉讼代理人：彭港，重庆周立太律师事务所律师。',
 '被告': '：重庆雅睿房地产营销策划有限公司，住所地重庆市渝北区吉乐大道48号3幢18-10，统一社会91500000MA5U4QMKXX。法定代表人：杨成，总监。',
 '案由+程序': '原告重庆市金德茧丝绸有限公司（以下简称金德公司）与被告重庆雅睿房地产营销策划有限公司（以下简称雅睿公司）房屋租赁合同纠纷一案，本院于2018年9月27日立案受理后，依法适用简易程序于2018年11月22日公开开庭进行了审理。原告金德公司的委托诉讼代理人彭港、被告雅睿公司的法定代表人杨成到庭参加诉讼。本案现已审理终结。',
 '诉讼请求': '金德公司向本院提出诉讼请求：1．被告支付2017年11月1日至2018年9月19日的租金70 087.67元、2017年5月至2018年9月的物业费、水电费及公摊费共计15 699.05元；2.被告支付逾期付款的违约金61 618.81元，计算方式为：以22 180元（2017年11月1日至2018年4月30日的欠付租金33 180元扣除保证金11 000元后的余额）为基数按日千分之五从2017年11月1日计算至2018年4月30日；以59 087.67元（2018年5月1日至2018年9月19日的欠付租金70 087.67元扣除保证金11 000元后）为基数按日千分之五从2018年5月1日计算至2018年9月19日；违约金总额为上述两项相加；3.解除《房屋租赁协议》；诉讼费由被告负担。',
 '事实理由': '：2017年4月27日，双方签订了租赁合同，原告将铺位出租给被告使用，然被告确欠付租金、物业费等。故为维护合法权益，遂起诉来院。',
 '适用法条': ''}

## step10:统计法官工作量并匹配法官（未在名单内的法官id为0）

In [54]:
def work(rootw,judge2id0,judge2id1,excel_data):
    gzl=os.path.join(rootw,'已计算工作量.csv')
    if os.path.exists(gzl):
        excel_data2_p=pd.read_csv(gzl)
    else:
#         excel_data=data.append(data_pre)
        excel_data2= pd.DataFrame(columns=excel_data.columns.values )#两方都在的案件信息
        drop_excel=[]#统计不在文书集中的案件
        kk=0
        for index, row in excel_data.iterrows():

            if row[2] not in keyDcitReplace:
                drop_excel.append(index);
            else:
                excel_data2.loc[kk] = row;
                kk=kk+1
        print(len(excel_data)-len(drop_excel)==len(excel_data2))#判断是否正确
        excel_data2_0=excel_data2[excel_data2['法院']=='重庆市江北区人民法院']
        excel_data2_0['法官id']=excel_data2_0['承办人'].apply(lambda x:judge2id0[x] if x in judge2id0 else 0)
        print('重庆市江北区人民法院处理完毕')
        excel_data2_1=excel_data2[excel_data2['法院']=='重庆市渝北区人民法院']
        excel_data2_1['法官id']=excel_data2_1['承办人'].apply(lambda x:judge2id1[x] if x in judge2id1 else 0)
        print('重庆市渝北区人民法院处理完毕')
        excel_data2_p=excel_data2_0.append(excel_data2_1);
        excel_data2_p['工作量']=0
        # temp['立案日期'].apply(lambda x:x.strftime("%Y-%m-%d %H:%M:%S"));datetime转字符串
        for index,row in excel_data2_p.iterrows():
            temp=excel_data2_p[excel_data2_p['法官id']==row[23]]
            excel_data2_p.loc[index,'工作量']=len(temp[(temp['立案日期']<=pd.to_datetime(row[11])) & (temp['结案日期']>pd.to_datetime(row[11]))])
    #   rootw=os.path.join(path,'数据清洗后','18_19民')
        excel_data2_p.to_csv(gzl,index=False) 
    return excel_data2_p

In [58]:
excel_data=data.append(data_pre)
rootw=os.path.join(path,'数据清洗后','18_19民')
excel_data2=work(rootw,judge2id0,judge2id1,excel_data)
len(excel_data2)

31501

In [62]:
data0=excel_data2[excel_data2['法院']=='重庆市江北区人民法院']
data1=excel_data2[excel_data2['法院']=='重庆市渝北区人民法院']

In [63]:
print(len(data0),len(data1))

12174 19327


## step11:案件打分

### 案件打分，输出格式：{'案号'：{'案由','案件得分','适用程序','实际时间','法官id','工作量'}}

In [64]:
def score(data0,anyoudict0):
    datadict={}
    for index, row in data0.iterrows():
        try:
            content={}

            content['案由']=row[3]
            a,b=plaintNum(row[4])
            c,d=plaintNum(row[5])
            num=anyoudict0[row[3]]
#             print(1)
            num=num+b+d+(a+c)/2
            num=num+money(row[8])
            num=num+proce(row[9])
            
            content['案件得分']=num
            content['实际时间']=int(row[21])
            content['适用程序']=row[9]
            content['法官id']=row[23]
            content['工作量']=row[24]
    #         content['案由得分']=anyoudict0(row[3])
    #         content['原告得分']=b
    #         content['被告得分']=d
    #         content['标的金额']=money(row[8])
    #         content['适用程序']=proce(row[9])
    #         content['主体类别']=(a+c)/2
            datadict[row[2]]=content
        except:
            
            continue
             
    print(len(datadict))
    return datadict

In [65]:
attribute0=score(data0,anyoudict0)
attribute1=score(data1,anyoudict1)

12174
19327


In [66]:
12174+19327

31501

In [69]:
attribute1['(2018)渝0112民初30233号']

{'案由': '房屋租赁合同纠纷',
 '案件得分': 5.5,
 '实际时间': 82,
 '适用程序': '简易程序（非小额诉讼）',
 '法官id': 464,
 '工作量': 29}

In [70]:
attribute=attribute0.update(attribute1)

In [73]:
attribute=attribute0

In [74]:
print(len(attribute))

31501


## step12: 合并信息

In [106]:
# 获取最终数据
def combineInfo(content,attribute,filename):
    '''
    content:读取文书集中的内容：{案号：{原告，被告，案由+程序,诉讼请求，事实理由，适用法条}，}
    :attribute: 同在文书集和excel中的案件列表:{案号:{'案由': '房屋租赁合同纠纷','案件得分': 5.5,'实际时间': 82, '适用程序': '简易程序（非小额诉讼）',
     '法官id': 464,'工作量': 29}，}
    :filename:将数据return之前，保存在filename文件中
    :return: 列表与json文件的组合：{'content': {'原告', '被告', '案由+程序', '诉讼请求', '事实理由'},
                                     'detail': {'案由':,'案件得分': ,'实际时间': , '适用程序': , '法官id': ,'工作量':}}
    '''
    exitNotread=[]
    if os.path.exists(os.path.join('../rawdata/数据清洗后/18_19民',filename)):
        read_file = open(os.path.join('../rawdata/数据清洗后/18_19民',filename), 'rb')
        data = pickle.load(read_file)
        read_file.close()
    else:
        high=0
        low=0
        data={}
        for key in attribute:
            if key in content:
                temp={}
                temp['content']=content[key]
                temp['detail']=attribute[key]
                if attribute[key]['案件得分']>=6:
                    high=high+1
                else:
                    low=low+1
                data[key]=temp
            else:
                exitNotread.append(key)
        output = open(os.path.join('../rawdata/数据清洗后/18_19民',filename), 'wb')
        pickle.dump(data, output,True)
        output.close()
        print('>=6',high,'<6',low)
        print('存在于excel和文书集但读取失败',len(exitNotread))
    return data

In [107]:
def repkh(content):
    content2={}
    for key in content:
        temp=content[key]
        key=key.replace('（','(')
        key=key.replace('）',')')
        content2[key]=temp
    print(len(content),len(content2))
    return content2


In [108]:
content2=repkh(content)
attribute2=repkh(attribute)

30654 30654
31501 31501


In [112]:
filename='datafinal.pkl'
datafinal=combineInfo(content2,attribute2,filename)#data,exitNotread

>=6 5878 <6 24776
存在于excel和文书集但读取失败 847


In [113]:
len(datafinal)#存在于excel和文书集但读取失败

30654

In [111]:
datafinal['(2018)渝0112民初30233号']

{'content': {'原告': '原告：重庆市金德茧丝绸有限公司，住所地重庆市万州区北山大道1323号，统一社会信用代码915001017428787877。法定代表人：彭中海，执行董事。委托诉讼代理人：彭港，重庆周立太律师事务所律师。',
  '被告': '：重庆雅睿房地产营销策划有限公司，住所地重庆市渝北区吉乐大道48号3幢18-10，统一社会91500000MA5U4QMKXX。法定代表人：杨成，总监。',
  '案由+程序': '原告重庆市金德茧丝绸有限公司（以下简称金德公司）与被告重庆雅睿房地产营销策划有限公司（以下简称雅睿公司）房屋租赁合同纠纷一案，本院于2018年9月27日立案受理后，依法适用简易程序于2018年11月22日公开开庭进行了审理。原告金德公司的委托诉讼代理人彭港、被告雅睿公司的法定代表人杨成到庭参加诉讼。本案现已审理终结。',
  '诉讼请求': '金德公司向本院提出诉讼请求：1．被告支付2017年11月1日至2018年9月19日的租金70 087.67元、2017年5月至2018年9月的物业费、水电费及公摊费共计15 699.05元；2.被告支付逾期付款的违约金61 618.81元，计算方式为：以22 180元（2017年11月1日至2018年4月30日的欠付租金33 180元扣除保证金11 000元后的余额）为基数按日千分之五从2017年11月1日计算至2018年4月30日；以59 087.67元（2018年5月1日至2018年9月19日的欠付租金70 087.67元扣除保证金11 000元后）为基数按日千分之五从2018年5月1日计算至2018年9月19日；违约金总额为上述两项相加；3.解除《房屋租赁协议》；诉讼费由被告负担。',
  '事实理由': '：2017年4月27日，双方签订了租赁合同，原告将铺位出租给被告使用，然被告确欠付租金、物业费等。故为维护合法权益，遂起诉来院。',
  '适用法条': ''},
 'detail': {'案由': '房屋租赁合同纠纷',
  '案件得分': 5.5,
  '实际时间': 82,
  '适用程序': '简易程序（非小额诉讼）',
  '法官id': 464,
  '工作量': 29}}

## 求某列字符串长度

In [263]:
test = {'Science':[1,2,3],'Art':[4,5,6]}

In [264]:
test1=pd.DataFrame(test)

In [265]:
test1.head(1)

,Science,Art
0,1,4


In [261]:
lens=test1.Science.str.len()

In [262]:
lens.max()

7.0